In [1]:
from collections import defaultdict
import numpy as np
import operator
import os
import pandas as pd
import pickle

import logging
logging.basicConfig(level=logging.DEBUG,
        format='%(levelname)-8s [%(lineno)d] %(message)s')

# Task 1: One-mode similarity

## Data: SimLex-999

In [2]:
simlex = pd.read_csv('/mnt/permanent/Language/English/Data/SimLex-999/SimLex-999.txt', sep='\t')

In [3]:
simlex.head()

,word1,word2,POS,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
0,old,new,A,1.58,2.72,2.81,2,7.25,1,0.41
1,smart,intelligent,A,9.20,1.75,2.46,1,7.11,1,0.67
2,hard,difficult,A,8.77,3.76,2.21,2,5.94,1,1.19
3,happy,cheerful,A,9.55,2.56,2.34,1,5.85,1,2.18
4,hard,easy,A,0.95,3.76,2.07,2,5.82,1,0.93


* conc(w1): The concreteness rating of word1 on a scale of 1-7. Taken from the University of South Florida Free Association Norms database. 

* conc(w2): The concreteness rating of word2 on a scale of 1-7. Taken from the University of South Florida Free Association Norms database. 

* concQ: The quartile the pair occupies based on the two concreteness ratings. Used for some analyses in the above paper. 

* Assoc(USF): The strength of free association from word1 to word2. Values are taken from the University of South Florida Free Association Dataset. 

* SimAssoc333: Binary indicator of whether the pair is one of the 333 most associated in the dataset (according to Assoc(USF)). This subset of SimLex999 is often the hardest for computational models to capture because the noise from high association can confound the similarity rating. See the paper for more details. 

* SD(SimLex): The standard deviation of annotator scores when rating this pair. Low values indicate good agreement between the 15+ annotators on the similarity value SimLex999. Higher scores indicate less certainty. 


In [5]:
simlex.describe(percentiles=[])

,SimLex999,conc(w1),conc(w2),concQ,Assoc(USF),SimAssoc333,SD(SimLex)
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,4.561572,3.657087,3.568629,2.501502,0.751512,0.333333,1.274505
std,2.614663,1.131050,1.159572,1.118145,1.344569,0.471641,0.366278
min,0.230000,1.190000,1.190000,1.000000,0.000000,0.000000,0.340000
50%,4.670000,3.830000,3.660000,3.000000,0.250000,0.000000,1.310000
max,9.800000,5.000000,5.000000,4.000000,8.850000,1.000000,2.180000


In [6]:
simlex.groupby('POS').size()

POS
A    111
N    666
V    222
dtype: int64

## Data: [SimVerb](http://people.ds.cam.ac.uk/dsg40/simverb.html) (Gerz+ EMNLP 2016)

In [20]:
simverb = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/simverb-3500/SimVerb-3500.txt', sep='\t',
                      header=None, names=['verb1', 'verb2', 'pos', 'sim', 'rel'])

In [3]:
simverb.describe(percentiles=[])

,sim
count,3500.000000
mean,4.291554
std,2.652621
min,0.000000
50%,4.320000
max,9.960000


In [4]:
simverb.head()

,verb1,verb2,pos,sim,rel
0,take,remove,V,6.81,SYNONYMS
1,walk,trail,V,4.81,COHYPONYMS
2,feed,starve,V,1.49,ANTONYMS
3,shine,polish,V,7.80,SYNONYMS
4,calculate,add,V,5.98,HYPER/HYPONYMS


In [5]:
simverb.groupby('rel').sim.describe(percentiles=[]).sort_values('count', ascending=False)

,count,mean,std,min,50%,max
rel,,,,,,
NONE,2093.0,3.431276,2.342695,0.0,3.150,9.79
HYPER/HYPONYMS,800.0,6.012525,2.104537,0.5,6.310,9.96
SYNONYMS,306.0,6.789150,2.104490,0.5,7.140,9.96
COHYPONYMS,190.0,4.435526,2.381992,0.0,4.665,9.30
ANTONYMS,111.0,0.977748,1.074232,0.0,0.660,6.04


## Testing the verb tensor

In [103]:
def sim_verb(weight, max_rank=256, cutoff=2, verb=True, normalize=True):
    # TODO verb=False debug
    _, index = pickle.load(open(
        '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/sparstensr_{}_{}.pkl'.format(weight, cutoff),
        mode='rb'))
    oov = defaultdict(int)
    target_df = simverb if verb else simlex
    target_cols = ['verb1', 'verb2'] if verb else ['word1', 'word2']
    relation = 'ROOT' if verb else 'nsubj'
    mode = 1 if verb else 0
    def tensor_based_sim(rank):
        ktensor, fit, n_iterations, exectimes = pickle.load(open(
            '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_{}_{}_{}.pkl'.format(
                weight, cutoff, rank),
            mode='rb'))
        #modes = ['nsubj', 'ROOT', 'dobj']
        if normalize:
            ktensor.U[mode] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[mode]).reshape((-1,1))
        def pointwise_prod_more(ser):
            try:
                return ktensor.lmbda.dot(
                    ktensor.U[mode][index[relation][ser[0]]] * ktensor.U[mode][index[relation][ser[1]]])
            except KeyError as e:
                oov[e.args] += 1
                return 0
        target_df['tensor_sim_{}'.format(rank)] = target_df[target_cols].apply(pointwise_prod_more, axis=1)
    #logging.debug(sorted(oov.items(), key=operator.itemgetter(1), reverse=True))
    for exp in range(1, int(np.log2(max_rank))+1):
        rank = 2**exp
        try:
            tensor_based_sim(rank=rank)
        except Exception as e:
            logging.warning(e)
            target_df['tensor_sim_{}'.format(rank)] = 0
    sim_col = 'sim' if verb else 'SimLex999'
    return target_df.corr().loc[sim_col][0 if verb else 7 :]

In [89]:
simverb.describe(percentiles=[])

,sim,tensor_sim_2,tensor_sim_4,tensor_sim_8
count,3500.000000,16.000000,14.000000,10.000000
mean,4.291554,0.279735,-inf,inf
std,2.652621,0.815630,NaN,NaN
min,0.000000,-0.941677,-inf,-1.062772
50%,4.320000,0.592328,0.036219,0.740908
max,9.960000,1.000461,5.231803,inf


In [104]:
simverb = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/simverb-3500/SimVerb-3500.txt', 
                      sep='\t', header=None, names=['verb1', 'verb2', 'pos', 'sim', 'rel'])
for weight in ['log_freq', 'pmi', 'iact_info', 'salience', 'iact_sali', 'log_dice']:
    print('\n{}'.format(weight))
    print(sim_verb(weight, cutoff=256))


log_freq


/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/home/makrai/tool/python/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
WARNING  [33] [Errno 2] No such file or directory: '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_log_freq_256_256.pkl'


sim               1.000000
tensor_sim_2      0.007131
tensor_sim_4     -0.026577
tensor_sim_8     -0.013559
tensor_sim_16    -0.006757
tensor_sim_32    -0.013412
tensor_sim_64     0.018288
tensor_sim_128   -0.007687
tensor_sim_256         NaN
Name: sim, dtype: float64

pmi


WARNING  [33] [Errno 2] No such file or directory: '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_pmi_256_256.pkl'


sim               1.000000
tensor_sim_2     -0.047096
tensor_sim_4     -0.047096
tensor_sim_8     -0.032358
tensor_sim_16    -0.013130
tensor_sim_32    -0.009400
tensor_sim_64     0.011003
tensor_sim_128    0.011259
tensor_sim_256         NaN
Name: sim, dtype: float64

iact_info


...

sim               1.000000
tensor_sim_2     -0.047096
tensor_sim_4     -0.047096
tensor_sim_8     -0.026716
tensor_sim_16    -0.022698
tensor_sim_32    -0.034623
tensor_sim_64    -0.015194
tensor_sim_128         NaN
tensor_sim_256         NaN
Name: sim, dtype: float64

iact_sali


WARNING  [33] [Errno 2] No such file or directory: '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_iact_sali_256_128.pkl'
WARNING  [33] [Errno 2] No such file or directory: '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_iact_sali_256_256.pkl'


sim               1.000000
tensor_sim_2     -0.005636
tensor_sim_4     -0.027604
tensor_sim_8      0.026368
tensor_sim_16    -0.012160
tensor_sim_32     0.005492
tensor_sim_64    -0.008530
tensor_sim_128         NaN
tensor_sim_256         NaN
Name: sim, dtype: float64

log_dice


WARNING  [33] [Errno 2] No such file or directory: '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_log_dice_256_128.pkl'
WARNING  [33] [Errno 2] No such file or directory: '/mnt/store/home/makrai/project/verb-tensor/just_svo/depCC/ktensor_log_dice_256_256.pkl'


sim               1.000000
tensor_sim_2      0.001052
tensor_sim_4     -0.000467
tensor_sim_8      0.007178
tensor_sim_16     0.022545
tensor_sim_32    -0.005034
tensor_sim_64     0.000570
tensor_sim_128         NaN
tensor_sim_256         NaN
Name: sim, dtype: float64


|weight |rank |sim |
|--|--|--|
|log(freq)|16|0.107064|
|pmi|8|0.107581|
|iact info|16|0.108490|
|salience|16|0.125190|

In [8]:
simverb.groupby('rel').corr().sim

rel                          
ANTONYMS        sim              1.000000
                tensor_sim_2     0.017516
                tensor_sim_4    -0.197749
                tensor_sim_8    -0.020607
                tensor_sim_16   -0.084390
                tensor_sim_32    0.037074
                tensor_sim_64         NaN
COHYPONYMS      sim              1.000000
                tensor_sim_2     0.043319
                tensor_sim_4     0.079444
                tensor_sim_8    -0.032908
                tensor_sim_16   -0.063021
                tensor_sim_32   -0.016319
                tensor_sim_64         NaN
HYPER/HYPONYMS  sim              1.000000
                tensor_sim_2    -0.007254
                tensor_sim_4    -0.002763
                tensor_sim_8    -0.043962
                tensor_sim_16    0.067686
                tensor_sim_32    0.026375
                tensor_sim_64         NaN
NONE            sim              1.000000
                tensor_sim_2     0.020203
    

# [Transitive sentence similarity](http://www.cs.ox.ac.uk/activities/compdistmeaning/GS2011data.txt) (Grefenstette and Sadrzadeh, EMNLP 2011)

## The dataset

In [9]:
svo_sim = pd.read_csv('/mnt/permanent/Language/English/Data/verb-similarity/GS2011data.txt', sep=' ', 
                 true_values=['HIGH'], false_values=['LOW'])

In [10]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant"]

In [11]:
svo_sim[cols_ordered].head()

,subject,verb,landmark,object,input,hilo,participant
0,family,provide,supply,home,4,True,participant20
1,government,provide,leave,cash,3,False,participant20
2,government,provide,supply,cash,7,True,participant20
3,man,provide,leave,money,6,True,participant20
4,man,provide,supply,money,7,True,participant20


In [12]:
svo_sim.describe(percentiles=[])

,input
count,2500.000000
mean,3.570800
std,2.193561
min,0.000000
50%,3.000000
max,7.000000


In [13]:
svo_sim.corr(method='spearman')

,input,hilo
input,1.00000,0.51689
hilo,0.51689,1.00000


## Testing the verb tensor

In [28]:
def predict_verb(weight, rank, cutoff=2, prec_at=5):
    _, index = pickle.load(open(
        '/mnt/store/home/makrai/project/verb-tensor/depCC/sparstensr_{}_{}.pkl'.format(weight, cutoff), mode='rb'))
    ktensor, fit, n_iterations, exectimes = pickle.load(open(
        '/mnt/store/home/makrai/project/verb-tensor/depCC/ktensor_{}_{}_{}.pkl'.format(weight, cutoff, rank), 
        mode='rb'))
    #modes = ['nsubj', 'ROOT', 'dobj']
    #ktensor.U[1] /= np.apply_along_axis(np.linalg.norm, 1, ktensor.U[1]).reshape((-1,1))
    oov = defaultdict(int)
    def verb_pred(ser):
        try:
            predicted_ids = np.argsort((
                -ktensor.lmbda * ktensor.U[0][index['nsubj'][ser[0]]]) .dot(
                (ktensor.U[1] * ktensor.U[2][index['dobj'][ser[0]]]).T))#] # 
            return [index['ROOT'].inverse[i] for i in predicted_ids[:prec_at]]
        except KeyError as e:
            oov[e.args] += 1
            return []
    svo_sim['predicted_{}_{}'.format(weight, rank)] = svo_sim[['subject', 'object']].apply(verb_pred, axis=1)
    #logging.debug(sorted(oov.items(), key=operator.itemgetter(1), reverse=True))
    for target in ['landmark', 'verb']:
        def is_good(ser):
            return ser[target] in ser['predicted_{}_{}'.format(weight, rank)]
        svo_sim['good_{}_{}_{}'.format(target, weight, rank)] = svo_sim.apply(is_good, axis=1)
        logging.debug((target, weight, rank, svo_sim['good_{}_{}_{}'.format(target, weight, rank)].sum()))

In [29]:
def for_weight(max_exp_plus_one=7):
    for weight in ['log_freq', 'pmi', 'iact_info', 'salience', 'log_dice']: # iact_sali
        #logging.info(weight)
        for exp in range(1, max_exp_plus_one):
            rank = 2 ** exp
            #logging.info(rank)
            predict_verb(weight, rank)

In [30]:
%time for_weight()

INFO     [3] log_freq
INFO     [6] 2
DEBUG    [25] 24
DEBUG    [25] 0
INFO     [6] 4
DEBUG    [25] 0
DEBUG    [25] 0
INFO     [6] 8
DEBUG    [25] 51
DEBUG    [25] 117
INFO     [6] 16
DEBUG    [25] 13
DEBUG    [25] 52
INFO     [6] 32
DEBUG    [25] 51
DEBUG    [25] 78
INFO     [6] 64
DEBUG    [25] 51
DEBUG    [25] 26
INFO     [3] pmi
INFO     [6] 2
DEBUG    [25] 0
DEBUG    [25] 0
INFO     [6] 4
DEBUG    [25] 0
DEBUG    [25] 74
INFO     [6] 8
DEBUG    [25] 13
DEBUG    [25] 48
INFO     [6] 16
DEBUG    [25] 0
DEBUG    [25] 74
INFO     [6] 32
DEBUG    [25] 0
DEBUG    [25] 144
INFO     [6] 64
DEBUG    [25] 26
DEBUG    [25] 168
INFO     [3] iact_info
INFO     [6] 2
DEBUG    [25] 0
DEBUG    [25] 0
INFO     [6] 4
DEBUG    [25] 0
DEBUG    [25] 74
INFO     [6] 8
DEBUG    [25] 0
DEBUG    [25] 96
INFO     [6] 16
DEBUG    [25] 25
DEBUG    [25] 148
INFO     [6] 32
DEBUG    [25] 12
DEBUG    [25] 168
INFO     [6] 64
DEBUG    [25] 13
DEBUG    [25] 144
INFO     [3] salience
INFO     [6] 2
DEBUG    [25] 24

CPU times: user 25min 36s, sys: 1h 37s, total: 1h 26min 13s
Wall time: 8min 3s


In [17]:
cols_ordered = ["subject", "verb", "landmark", "object", "input", "hilo", "participant", 'predicted', 'good']

In [31]:
for target in ['landmark', 'verb']:
    print(target)
    print(svo_sim.groupby(target).size().describe(percentiles=[]))

landmark
count     20.000000
mean     125.000000
std        4.588315
min      120.000000
50%      125.000000
max      130.000000
dtype: float64
verb
count     10.00000
mean     250.00000
std        9.42809
min      240.00000
50%      250.00000
max      260.00000
dtype: float64


In [34]:
svo_sim.sum(numeric_only=True).sort_values(ascending=False)/svo_sim.shape[0]

input                         3.5708
hilo                          0.4652
good_verb_pmi_64              0.0672
good_verb_iact_info_32        0.0672
good_verb_iact_info_16        0.0592
good_verb_pmi_32              0.0576
good_verb_iact_info_64        0.0576
good_verb_salience_64         0.0512
good_verb_log_freq_8          0.0468
good_verb_salience_8          0.0468
good_verb_log_dice_64         0.0416
good_verb_iact_info_8         0.0384
good_verb_log_dice_32         0.0312
good_verb_salience_16         0.0312
good_verb_log_freq_32         0.0312
good_verb_log_dice_16         0.0304
good_verb_pmi_4               0.0296
good_verb_pmi_16              0.0296
good_verb_iact_info_4         0.0296
good_verb_log_freq_16         0.0208
good_landmark_log_freq_64     0.0204
good_landmark_salience_8      0.0204
good_landmark_log_freq_32     0.0204
good_landmark_log_freq_8      0.0204
good_verb_log_dice_8          0.0200
good_verb_pmi_8               0.0192
good_landmark_log_dice_32     0.0192
g

|verb|`* 1`|`* lmbda`|
|----|--|------|
|unnorm|130|**272**|
|norm|0|24|

Majoroty baseline...

In [38]:
target = 'landmark'
svo_sim.groupby(target).size().sort_values()/len(svo_sim[target].unique())

landmark
allege      6.00
attract     6.00
bear        6.00
depict      6.00
express     6.00
state       6.00
picture     6.00
receive     6.00
bribe       6.25
supply      6.25
leave       6.25
purchase    6.25
spell       6.50
satisfy     6.50
operate     6.50
test        6.50
move        6.50
judge       6.50
publish     6.50
visit       6.50
dtype: float64

# [Verb prediction (tensor)](https://everest.hds.utc.fr/doku.php?id=en:lfmnips12) (Jenatton+ NIPS 2012)

In [ ]:
dataset_dir = '/mnt/permanent/Language/English/Data/verb-similarity/SVO-tensor-dataset/'
def get_index(pos):
    with open(os.path.join(dataset_dir, 'svo-{}s.lst'.format(pos))) as infile:
        return {i+1: ' '.join(line.strip().split('_')[2:-1]) for i, line in enumerate(infile)}

In [ ]:
svo_df = pd.read_csv(os.path.join(dataset_dir, 'svo_data_train_1000000.dat'), sep='\t', header=None, 
                     names=['subject', 'verb', 'object'])

In [ ]:
index_verb = get_index('verb')
index_noun = get_index('noun')

In [ ]:
svo_df.subject = svo_df.subject.apply(index_noun.get)
svo_df.verb = svo_df.verb.apply(index_verb.get)
svo_df.object = svo_df.object.apply(index_noun.get)

In [ ]:
svo_df.head(10)